# Revised case normalization for Hirslanden Zurich 2017


In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
        'Hirslanden Zurich',
        '2017',
        ['Änderungen_Hirslanden_2017'])
print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Zurich', year='2017', sheets=['Änderungen_Hirslanden_2017'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-17 14:01:18.797 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 106: discarded ICDs after validation {''}
2022-10-17 14:01:18.797 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 110: discarded ICDs after validation {''}


Read 128 cases for Hirslanden Zurich 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


2022-10-17 14:01:18.800 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 0: discarded ICDs after validation {''}
2022-10-17 14:01:18.801 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 1: discarded ICDs after validation {''}
2022-10-17 14:01:18.801 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-17 14:01:18.802 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 4: discarded ICDs after validation {''}
2022-10-17 14:01:18.803 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 5: discarded ICDs after validation {''}
2022-10-17 14:01:18.804 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 6: discarded ICDs after validation {''}
2022-10-17 14:01:18.805 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 7: discarded ICDs after validation {''}
2022-10-17 14:01:18.805 | DEBUG    | src.utils.dataframe_utils:_validate_icd

In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,41230054,E8CD6759509644DE,M,78,11,3,E06C,C341,C341,M200,[J939],[],[],[]
1,41194118,41309F148FEDBFD2,W,39,7,3,O01E,O335,O335,M300,[K560],[],[],[]
2,41230059,7E79B28620A3A8E3,M,84,8,3,M09B,C61,C61,M200,[C795],[],[],[]
3,41231720,FBB0408939EC3E8B,M,81,6,3,E65C,J4402,J4402,M100,[E1191],[E1190],[],[]
4,41243610,32E2ED09CA696444,W,82,7,3,E65C,J4412,J4412,M100,[J81],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,41249123,9F60C5E2FF49ADFA,M,72,9,2,L18B,C671,C671,M200,[U804],[],[],[]
124,41323252,473038E4266105E6,W,80,8,3,L06B,N390,N390,M100,[U804],[],[],[]
125,41340499,38988935DC3B2582,M,57,7,3,L63D,N10,N10,M100,[U804],[U81],[],[]
126,41347960,4CA999AAFA0018CC,W,76,3,0,E65C,J4480,J4480,M100,[],[],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['41230054', '41194118', '41230059', '41231720', '41243610',
       '41250259', '41259511', '41266263', '41279571', '0041284390',
       '41284502', '41304546', '41305897', '41313648', '41317440',
       '41323906', '41211150', '0041221789', '41240445', '41273637',
       '41383051', '41389189', '41391650', '41399129', '41399319',
       '41400362', '0041409899', '41410139', '41419224', '41419713',
       '41420006', '41428184', '41323757', '41332064', '41332184',
       '41333430', '41346910', '0041360825', '41361041', '41381268',
       '41446034', '41446034', '41447020', '41447192', '41455138',
       '41455219', '41462019', '41466057', '41468530', '41475302',
       '41475490', '41475511', '41480679', '41483323', '41381986',
       '41382245', '41400870', '41401698', '41402639', '41501599',
       '41486629', '41487907', '41495097', '41505482', '41506204',
       '41506923', '41527604', '41530903', '41532398', '41546799',
       '41543987', '41559160', '41560984', '41566233',

In [6]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

In [7]:
# change the case_id the same format as bfs data e.g. '0041194601' 
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(revised_case_id_stripped)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,1.470,46629,1,0041230054,E8CD6759509644DE,78,0,M,11,4,0,0,0,2017-01-17,01,2017-01-28,00,E06C,E06,3
0,1.033,45907,1,0041194118,41309F148FEDBFD2,39,0,W,7,5,0,0,0,2017-02-09,01,2017-02-16,00,O01E,O01,3
0,0.809,46630,1,0041230059,7E79B28620A3A8E3,84,0,M,8,4,0,0,0,2017-01-06,01,2017-01-14,00,M09B,M09,3
0,0.714,46734,1,0041231720,FBB0408939EC3E8B,81,0,M,6,3,0,0,0,2016-12-27,01,2017-01-02,00,E65C,E65,3
0,0.714,47423,1,0041243610,32E2ED09CA696444,82,0,W,7,3,0,0,0,2017-01-10,01,2017-01-17,00,E65C,E65,3


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped
0,1.470,46629,1,0041230054,E8CD6759509644DE,78,0,M,11,4,...,0,0,2017-01-17,01,2017-01-28,00,E06C,E06,3,41230054
0,1.033,45907,1,0041194118,41309F148FEDBFD2,39,0,W,7,5,...,0,0,2017-02-09,01,2017-02-16,00,O01E,O01,3,41194118
0,0.809,46630,1,0041230059,7E79B28620A3A8E3,84,0,M,8,4,...,0,0,2017-01-06,01,2017-01-14,00,M09B,M09,3,41230059
0,0.714,46734,1,0041231720,FBB0408939EC3E8B,81,0,M,6,3,...,0,0,2016-12-27,01,2017-01-02,00,E65C,E65,3,41231720
0,0.714,47423,1,0041243610,32E2ED09CA696444,82,0,W,7,3,...,0,0,2017-01-10,01,2017-01-17,00,E65C,E65,3,41243610


In [10]:
df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()


,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped
0,41230054,E8CD6759509644DE,M,78,11,3,E06C,C341,C341,M200,[J939],[],[],[],41230054
1,41194118,41309F148FEDBFD2,W,39,7,3,O01E,O335,O335,M300,[K560],[],[],[],41194118
2,41230059,7E79B28620A3A8E3,M,84,8,3,M09B,C61,C61,M200,[C795],[],[],[],41230059
3,41231720,FBB0408939EC3E8B,M,81,6,3,E65C,J4402,J4402,M100,[E1191],[E1190],[],[],41231720
4,41243610,32E2ED09CA696444,W,82,7,3,E65C,J4412,J4412,M100,[J81],[],[],[],41243610


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [11]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(128, 28)


/tmp/ipykernel_11876/3233428173.py:4: FutureWarning: Passing 'suffixes' as a <class 'NoneType'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)


,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped,gender,age_years,...,age_days,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,adrg
0,C341,C341,M200,[J939],[],[],[],41230054,M,78,...,0,4,0,0,0,2017-01-17,01,2017-01-28,00,E06
1,O335,O335,M300,[K560],[],[],[],41194118,W,39,...,0,5,0,0,0,2017-02-09,01,2017-02-16,00,O01
2,C61,C61,M200,[C795],[],[],[],41230059,M,84,...,0,4,0,0,0,2017-01-06,01,2017-01-14,00,M09
3,J4402,J4402,M100,[E1191],[E1190],[],[],41231720,M,81,...,0,3,0,0,0,2016-12-27,01,2017-01-02,00,E65
4,J4412,J4412,M100,[J81],[],[],[],41243610,W,82,...,0,3,0,0,0,2017-01-10,01,2017-01-17,00,E65


In [12]:
df_revised_case_normalized = revised_case_normalized[COLS_TO_SELECT]
df_revised_case_normalized.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,0041230054,E8CD6759509644DE,M,78,11,3,E06C,C341,C341,M200,[J939],[],[],[]
1,0041194118,41309F148FEDBFD2,W,39,7,3,O01E,O335,O335,M300,[K560],[],[],[]
2,0041230059,7E79B28620A3A8E3,M,84,8,3,M09B,C61,C61,M200,[C795],[],[],[]
3,0041231720,FBB0408939EC3E8B,M,81,6,3,E65C,J4402,J4402,M100,[E1191],[E1190],[],[]
4,0041243610,32E2ED09CA696444,W,82,7,3,E65C,J4412,J4412,M100,[J81],[],[],[]


# Export the normalized revised cases with the neccessary columns to a csv file


In [13]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [14]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are 128 out of 128 revised cases from DtoD that can be matched with the bfs database data for Hirslanden Zurich 2017


In [15]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))

All the normalized revised cases are exported to here: /home/jovyan/work/src/revised_case_normalization/normalized_revision_data/Hirslanden_Zurich_2017.csv
